In [1]:
from models.base_model import GMF

import pandas as pd

In [2]:
sessions_df = pd.read_json('data_files/sessions.jsonl', lines=True)

In [3]:
unique_users_count = sessions_df['user_id'].nunique()
unique_tracks_count = sessions_df['track_id'].nunique()
print(unique_users_count, unique_tracks_count)

3000 12303


In [10]:
from models.training import GMFTrainer

sessions_train_df = pd.read_json('data_files/train_sessions.jsonl', lines=True)

model = GMF(unique_users_count, unique_tracks_count)
gmf_trainer = GMFTrainer(